# Qi's Covid Dashboard
This dashboard shows the growth in number of accumulative cases and  accumulative tests.



In [8]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [9]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [10]:
# Load JSON files and store the raw data in some variable. Edit as appropriate

#with open("comparision2nd.json", "rt") as INFILE:
    #jsondata=json.load(INFILE)
with open("comparision2nd.json", "r") as INFILE:
    jsondata=json.loads(INFILE.read())

In [11]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    df = pd.json_normalize(jsondata, record_path = ['data'])
    df = df.loc[::-1].reset_index(drop=True)
    df.fillna(0.0, inplace=True)

    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting
#df.head()

In [12]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=nation',
        'areaName=England'
    ]


# values here are the names of the PHE metrics
    structure = {
        "date": "date",
        "test": "cumPCRTestsByPublishDate",
        "admission": "cumAdmissions"
    }

    api = Cov19API(filters=filters, structure=structure)
    comparision2nd=api.get_json()
    return comparision2nd # return data read from the API

In [23]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='REFRESH', # you may want to change this...
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='check'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button


def plot_comparision2nd(graphcolumns):
    plt.plot(df['date'],df[graphcolumns])
    plt.xticks(np.arange(0,980,30), rotation=45)
    plt.show() # important! update won't work properly without this
    
cols=wdg.Dropdown(
    options=['test', 'admission'],
    value='test',
    description='Option: ',
    disabled=False,
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([cols, scale])

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=cols.value
    if current==cols.options[0]:
        other=cols.options[1]
    else:
        other=cols.options[0]
    cols.value=other # forces the redraw
    cols.value=current # now we can change it back
    
    
graph=wdg.interactive_output(plot_comparision2nd, {'graphcolumns': cols})
    
display(cols, graph, scale)

Button(button_style='success', description='REFRESH', icon='check', style=ButtonStyle(), tooltip='Keep calm an…

Dropdown(description='Option: ', options=('test', 'admission'), value='test')

Output()

RadioButtons(description='Scale:', options=('linear', 'log'), value='linear')